# Validating fine-tuned Generator via fine-tuned Classifier

In [8]:
# setup 
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import pprint
import pandas as pd
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
    balanced_accuracy_score
)
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import numpy as np
from tqdm import tqdm
from typing import List, Dict, Any
from datasets import Dataset


# load data
generated_speeches_df = pd.read_csv("../data/generated_speeches_final.csv")



In [2]:
generated_speeches_df

,party,topic,speech_base_model_01,speech_ft_model_01,speech_base_model_03,speech_ft_model_03,RAG_speech_01,RAG_speech_03
0,Union,Mindestlohn,#Hartz-IV# - Ein neuer Start für den Arbeitsma...,#NieAgain – Sexuelle Gewalt an Kindern verhind...,#HartzIV - Die Zeit der Verantwortung für alle...,# Hochwertige Arbeit für alle – Ein guter Job ...,kurz kommentieren. Es geht dort tatsächlich um...,kurz kommentieren. Es geht dort tatsächlich um...
1,Union,Bundeswehreinsatz im Kosovo,#Hallo! Ich bin der Kanzlerminister in dieser ...,#NieWieder – Das war der Slogan des Protestes ...,#Hallo! Ich bin der Kanzlerminister in dieser ...,#NieJamaisKrieg – so lautet der Slogan des Fri...,[Inst] Sehr verehrte Frau Präsidentin! Lieber ...,[Inst] Sehr verehrte Frau Präsident! Lieber Ko...
2,Union,Wirtschaftshilfen Corona,#Hallo! Ich bin der Kanzlerminister Olaf Schul...,#NieAgain – so lautet der Slogan des internati...,#Hallo! Ich bin der Kanzlerminister Olaf Schul...,#NieAgain – wir müssen lernen aus den Fehlents...,"Bildungsfördersprechanbieter, also Förster/-in...",Bildungsfördermittel überzeugt gewesen. Doch d...
3,Union,Gaspreise,#Hallo! Ich bin der Kanzlerminister in dieser ...,#NieAgain! – Das war der Slogan des großen Pro...,#Hallo! Ich bin der Kanzlerminister in dieser ...,#NieAgain! Wir werden unsere Energie unabhängi...,"Kommission versuchen, diesen Decker möglichst ...",Kommission diesen Decker durchsetzten sowie we...
4,SPD,Mindestlohn,"#HartzIV#Mindesteinkommen#, Ladies and Gentlem...",#NieAufKeinenFall! – Das war der Aufruf des Bu...,#HartzIV - Die soziale Marktwirtschaft braucht...,#NieAufKeinenFall – Das muss endlich Stoppen! ...,[Inst] Sehr verehrte Präsidentin! Meine geschä...,[Inst] Sehr verehrte Präsidentin! Meine geschä...
5,SPD,Bundeswehreinsatz im Kosovo,#Hallo! Ich bin der Kanzlerminister Olaf Schol...,import { default as Spinner } from 'react-spin...,#Hallo! Ich bin hier heute als Vertreterin der...,#NieAgain! Kein Krieg mehr für Deutschland – K...,damit maßgeblich zur Unterbindung potentieller...,damit maßgeblich zur Unterbindung potentieller...
6,SPD,Wirtschaftshilfen Corona,#WirHelfendeNation# - Eine solide wirtschaftli...,#NieAgain – Das Ziel der Bundesregierung muss ...,#WirHelfendeNation# - Eine solide wirtschaftli...,#NieAgain! – Das war der Slogan des Protestes ...,Landes sowie seine Standortattraktivität erhal...,Landes sowie seiner Gesellschaft trotz schwers...
7,SPD,Gaspreise,#Hallo! Ich bin der Kanzlerminister Olaf Schol...,#NieAgain! Wir müssen den Krieg in der Ukraine...,#WirfürEnergie - Eine nachhaltige Energiesyste...,#NieAgain – wir müssen endlich den Krieg in de...,wir fortsetzten. Zum Ende meiner Redezeit würd...,"wir nun begreifen, und darüber hinaus wollen w..."
8,GRÜNE,Mindestlohn,#Mindesteinkommen für alle!# Ein neuer Start i...,"#Mindesteinkommen#Minijob#, wir alle wissen es...","#HartzIV#Mindesteinkommen#ArbeitslosengeldII#,...",#NieMindeloesung! Das hat der Kollege Birkwald...,[Inst] Sehr verehrte Frau Präsidentin! Lieber ...,[INST] Sehr verehrte Frau Präsidentin! Lieber ...
9,GRÜNE,Bundeswehreinsatz im Kosovo,#Bundeswehr#Kosovokrieg#Politikdebatte#GrüneFr...,#NieWieder! – Das war der Aufruf des Klimaprot...,"#Bundeswehr#Kosovokrieg"" class=""tw-breadcrumbs...",#NieWiederKosovowar! – Das war der Aufruf des ...,zu würdigen. Als letzte Gedenkmöglichkeit will...,kontinuierlich würdigen zu lassen. Insofern bi...


In [ ]:
# load best classifier model

# for now second best

# Load model and tokenizer from local folder
model = AutoModelForSequenceClassification.from_pretrained("../data/allresultsA/")
tokenizer = AutoTokenizer.from_pretrained("../data/allresultsA/")


/Users/ellaalle/anaconda3/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/ellaalle/anaconda3/lib/python3.11/site-packages/torchvision/image.so, 6): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: /Users/ellaalle/anaconda3/lib/python3.11/site-packages/torchvision/image.so
  Expected in: /Users/ellaalle/anaconda3/lib/python3.11/site-packages/torch/lib/libtorch_cpu.dylib
 in /Users/ellaalle/anaconda3/lib/python3.11/site-packages/torchvision/image.so'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
2025-08-04 10:05:04.422478: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To en

In [ ]:
# define parameters
WINDOW_LENGTH = 512
STRIDE = 256


label_names = ['CDU/CSU', 'SPD', 'GRÜNE', 'FDP', 'AfD', 'LINKE']
label2id = {label: i for i, label in enumerate(sorted(label_names))}
id2label = {i: label for label, i in label2id.items()}

In [ ]:
def sliding_window_tokenize(batch):
    texts = batch["speech_text"]
    labels = batch["label"]  # ensure this is a flat list of ints

    tokenized = tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=WINDOW_LENGTH,
        stride=STRIDE,
        return_overflowing_tokens=True,
    )

    # Assign each overflow window the correct label
    tokenized["labels"] = [label2id[labels[i]] for i in tokenized["overflow_to_sample_mapping"]]

    return tokenized


In [ ]:

# model und tokenizer müssen schon geladen sein
model.eval()

def tokenize_sliding_windows(example: Dict[str, Any]) -> Dict[str, Any]:
    encoding = tokenizer(
        example["speech_text"],
        truncation=True,
        padding="max_length",
        max_length=WINDOW_LENGTH,
        stride=STRIDE,
        return_overflowing_tokens=True,
        return_offsets_mapping=False,
        return_tensors="pt"
    )
    return encoding

def predict_proba_for_dataset(dataset: Dataset) -> List[Dict[str, Any]]:
    results = []

    for example in tqdm(dataset):
        tokenized = tokenize_sliding_windows(example)
        input_ids = tokenized["input_ids"].to(model.device)
        attention_mask = tokenized["attention_mask"].to(model.device)

        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            probs = torch.nn.functional.softmax(outputs.logits, dim=-1).cpu().numpy()

        avg_probs = probs.mean(axis=0)
        results.append({
            "probs": avg_probs.tolist(),
            "label": example["label"]  # falls du wahren Wert mitgeben willst
        })

    return results


BertTokenizerFast(name_or_path='../data/allresultsA/', vocab_size=30000, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	5: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [ ]:
results_val_A = predict_proba_for_dataset(val_data, sorted(label_names))